# Python Intro - Final Exercise


In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [2]:
import pyodbc 
conn = pyodbc.connect("DSN=college;"
                      "server=DESKTOP-F1Q7Q52\SQLEXPRESS;"
                      "Database=college;"
                      "Trusted_Connection=yes;")




In [3]:
cursor = conn.cursor()
cursor.execute('SELECT * FROM Departments')

for row in cursor:
    print('row = %r' % (row,))

row = (1, ' English ')
row = (2, ' Science ')
row = (3, ' Arts ')
row = (4, ' Sport ')


In [4]:
students = pd.read_sql("SELECT * FROM Students;",conn) 
classrooms = pd.read_sql("SELECT * FROM Classroom;",conn) 
courses = pd.read_sql("SELECT * FROM Courses;",conn) 
departments = pd.read_sql("SELECT * FROM Departments;",conn) 
teachers = pd.read_sql("SELECT * FROM Teachers;",conn) 

students.head()




,StudentId,FirstName,LastName,Gender
0,162,Elmira,Holston,F
1,154,Sierra,Van Vorst,F
2,28,Mycah,Bingham,F
3,54,Alia,Jones,F
4,31,Chanda,Fillmore,F


In [5]:
data1= pd.merge( classrooms  ,courses,how='left', left_on=['CourseId'], right_on = ['CourseId'])



In [6]:
data2 = pd.merge( students,data1,how='right',left_on=['StudentId'], right_on = ['StudentId'])
data2.head()

,StudentId,FirstName,LastName,Gender,CourseId,degree,CourseName,DepartmentID,TeacherId
0,162,Elmira,Holston,F,8,66.0,Chemistry,2,14
1,162,Elmira,Holston,F,20,79.0,History of Arts,3,8
2,162,Elmira,Holston,F,31,75.0,Gymnastics,4,23
3,154,Sierra,Van Vorst,F,6,84.0,Algebra,2,13
4,154,Sierra,Van Vorst,F,18,81.0,R,2,17


In [7]:
data3 =pd.merge(data2,departments, left_on=['DepartmentID'], right_on = ['DepartmentId'])


In [8]:
Data_All = pd.merge(data3,teachers,left_on=['TeacherId'], right_on = ['TeacherID'])


In [9]:
Data_All.head()

,StudentId,FirstName_x,LastName_x,Gender_x,CourseId,degree,CourseName,DepartmentID,TeacherId,DepartmentId,DepartmentName,TeacherID,FirstName_y,LastName_y,Gender_y
0,162,Elmira,Holston,F,8,66.0,Chemistry,2,14,2,Science,14,Jeffrey,Batzel,M
1,110,Ashleigh,Conder,F,8,82.0,Chemistry,2,14,2,Science,14,Jeffrey,Batzel,M
2,36,Sarah,Sampson,F,8,80.0,Chemistry,2,14,2,Science,14,Jeffrey,Batzel,M
3,234,Adrianna,Bean,F,8,67.0,Chemistry,2,14,2,Science,14,Jeffrey,Batzel,M
4,187,Sabrina,Mcclure,F,8,79.0,Chemistry,2,14,2,Science,14,Jeffrey,Batzel,M


# Questions
 
 
## Q1. Count the number of students on each department

In [10]:
Data_All1 = Data_All[['DepartmentName', 'StudentId']].drop_duplicates().groupby('DepartmentName').agg({'StudentId':['count']})
Data_All1

,StudentId
,count
DepartmentName,
Arts,137
English,84
Science,200
Sport,81


# Q2. How many students have each course of the English department and the total number of students in the department?

In [64]:
DataEng = Data_All.loc[(Data_All['DepartmentId'].isin([1,]))]

DataEng.groupby(['CourseName'])['StudentId'].agg('count')



CourseName
 Advanced English       24
 English Begginers      32
 ProffesionalEnglish    38
Name: StudentId, dtype: int64

In [65]:

DataEng.drop_duplicates().groupby(['DepartmentName'])['StudentId'].agg('count')

DepartmentName
 English     94
Name: StudentId, dtype: int64

# Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [101]:
DataScie =Data_All


DataScie ['Res'] = 1

DataScie = Data_All.loc[(Data_All['DepartmentId'].isin([2,]))]

DataScie = DataScie.groupby('CourseId').Res.agg('count')


print("Number of small classrooms is : ",DataScie[DataScie < 22].shape[0])

print("Number of large classrooms  is: ",DataScie[DataScie >= 22].shape[0])



Number of small classrooms is :  6
Number of large classrooms  is:  10


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [25]:
students = students.groupby('Gender').agg({'StudentId':['count']})
students

,StudentId
,count
Gender,
F,165
M,115


## Q5. For which courses the percentage of male/female students is over 70%?

In [143]:
Data_All['Female'] = 0
Data_All['Male'] = 0


Data_All.loc[Data_All.Gender_x == ' F','Female'] = 1
Data_All.loc[Data_All.Gender_x == ' M','Male'] = 1

ratio = Data_All.groupby('CourseName').agg({'Female':['mean']})
if (ratio[ratio < 0.7].dropna().shape[1] ):
   print("Courses with Female grater than 70%: \n", ratio[ratio > 0.7].dropna())

ratio = Data_All.groupby('CourseName').agg({'Male':['mean']})
if (ratio[ratio > 0.7].dropna().shape[0] > 0 ):
    print("Courses with males grater than 70%: \n", ratio[ratio > 0.7].dropna())

    
    

Courses with Female grater than 70%: 
                Female
                 mean
CourseName           
 Sculpture   0.708333
 Tenis       0.722222


## Q6. For each department, how many students passed with a grades over 80?

In [312]:
A_Student = Data_All[['DepartmentName', 'StudentId', 'degree']]

A_Student = A_Student[A_Student['degree'] > 80].drop_duplicates()

A_Student.groupby('DepartmentName')['StudentId'].agg('count')

DepartmentName
 Arts         63
 English      41
 Science     156
 Sport        35
Name: StudentId, dtype: int64

## Q7. For each department, how many students passed with a grades under 60?

In [313]:
D_Student = Data_All[['DepartmentName', 'StudentId', 'degree']]

D_Student = D_Student[D_Student['degree'] < 60].drop_duplicates()

D_Student.groupby('DepartmentName')['StudentId'].agg('count')

DepartmentName
 Arts        3
 English     2
 Science     6
 Sport       1
Name: StudentId, dtype: int64

## Q8. Rate the teachers by their average student's grades (in descending order).

In [315]:
Data_All.head()

Data_All.groupby(['FirstName_y','LastName_y','TeacherId'])['degree'].agg('mean').sort_values(ascending=False)


FirstName_y   LastName_y    TeacherId
  Darick       Hess         22           81.187500
  Jordyn       Baron        3            80.722222
  Heather      Metcalf      5            80.500000
  Kaylea       Monzingo     1            79.342105
  Zachary      Healy        20           79.333333
  Daniel       Bland        18           78.950000
  Keegan       Price        24           78.777778
  Maxwell      Romkee       23           78.708333
  Conor        Mckittrick   15           78.702128
  Terran       Beaty        19           78.476744
  Mackenzie    Olsen        10           78.466667
  Tanner       Fitzsimons   17           78.411765
  Ernest       Stoyanov     21           78.269231
  Madison      Batan        6            77.733333
  Katerina     Young        9            77.382353
  Mikayla      Russell      12           77.055556
  Amanda       Dworkin      2            76.833333
  Jeffrey      Batzel       14           76.791667
  Kaitlin      Delude       11           76.

## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [316]:
Data_All.groupby(['CourseName','DepartmentName','FirstName_y','LastName_y'])['StudentId'].agg('count')


CourseName            DepartmentName  FirstName_y   LastName_y  
 Advanced English      English          Conor        Mckittrick     24
 Algebra               Science          Alexander    Watts          12
 Biology               Science          Jordyn       Baron          36
 Chemistry             Science          Jeffrey      Batzel         24
 Computer Science      Science          Kaylea       Monzingo       38
 Data Science          Science          Mackenzie    Olsen          30
 English Begginers     English          Conor        Mckittrick     32
 Genetics              Science          Terran       Beaty          38
 Geometry              Science          Alexander    Watts          14
 Gymnastics            Sport            Maxwell      Romkee         24
 History of Arts       Arts             Caroline     Knuesel        32
 Java                  Science          Darick       Hess           16
 Literature            Arts             Daniel       Bland          40
 Matematics 

## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [145]:
Data_All['Student_Grade_Avg']=0

AVG=Data_All.groupby(['StudentId'])['degree'].agg(['mean', 'count']).reset_index()

AVG.columns = ['StudentId', 'Total_Degree', 'Number_Of_Courses']
STU_AVG = Data_All[['StudentId', 'FirstName_x','LastName_x', 'DepartmentName', 'degree']]

STU_AVG = pd.pivot_table(STU_AVG, values='degree',
                     index=['StudentId', 'FirstName_x','LastName_x'],
                     columns='DepartmentName',
                     aggfunc=['mean']).reset_index()

STU_AVG.columns = ['StudentId', 'FirstName_x','LastName_x','English_Degree','Science_Degree', 'Arts_Degree','Sport_Degree']

STU_AVG = pd.merge(avg, STU_AVG, on='StudentId')

STU_AVG = STU_AVG.drop(['StudentId'], axis=1)

cols = STU_AVG.columns.tolist()

cols = cols[2:] + cols[:2]

STU_AVG = STU_AVG[cols]

STU_AVG

,FirstName_x,LastName_x,English_Degree,Science_Degree,Arts_Degree,Sport_Degree,Total_Degree,Number_Of_Courses
0,Alexandria,Heller,87.000000,NaN,79.333333,NaN,81.250000,4
1,Cassie,Liska,79.500000,NaN,78.666667,NaN,79.000000,5
2,Hayley,Frank,NaN,84.0,76.000000,NaN,78.666667,3
3,Maxwell,Kotch,69.000000,71.0,74.000000,NaN,71.333333,3
4,Dylan,Boehm,64.000000,NaN,100.000000,66.0,76.666667,3
5,Chelsea,Vuolo,69.000000,NaN,74.000000,NaN,71.500000,2
6,Katrina,Evers,NaN,NaN,73.250000,NaN,73.250000,4
7,Sean,Hasenack,96.000000,89.0,79.000000,NaN,88.000000,3
8,Madalyn,Miller,NaN,NaN,87.000000,NaN,87.000000,2
9,Katrina,Miller,80.000000,NaN,78.500000,85.0,80.500000,4
